In [22]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris

In [104]:
features, target = load_iris(return_X_y=True, as_frame=True)
features.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Удалим из стратифицированной выборки треть данных

In [106]:
kfold = StratifiedKFold(n_splits=3, shuffle=False)
for ((train_index, test_index), index) in zip(kfold.split(features, target), range(0, 5)):
    if index == 1:
        features.loc[test_index] = np.nan

features.isna().sum()

sepal length (cm)    50
sepal width (cm)     50
petal length (cm)    50
petal width (cm)     50
dtype: int64

1. Обучить модель по бейзлайну, считать такую модель эталонной
2. Удалить часть данных из стратифицированной выборки
3. Восстановить данные методом ручной группировки и проверить точность модели из бейзлайна
4. Восстановить с помощью IterativeImputer и проверить точность модели из бейзлайна